In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from space_env import Satellite, Propagator

print("Entorno cargado correctamente.")

In [ ]:
# Instanciamos el satelite (esto crea el objeto Spacecraft en GMAT)
sat = Satellite("Saocom")

# Configuramos una orbita circular a 700 km de altura (LEO)
# SMA = 6371 (Radio Tierra) + 700 = 7071 km
sat.set_keplerian(
    sma=7071, 
    ecc=0.0001, 
    inc=98.2, 
    raan=0, 
    aop=0, 
    ta=0
)

print(f"Satelite '{sat.name}' configurado.")

In [ ]:
# 1. Definir la fidelidad de la física
config_leo = {
    'gravity': 'Earth',
    'degree': 2,    # Activa J2
    'order': 0,
    'drag': True    # Resistencia atmosférica (importante en LEO)
}

# 2. Instanciar el propagador (esto crea el ForceModel y el Propagator en GMAT)
prop = Propagator("MyPropagator", config=config_leo)

In [ ]:
# 5 horas en segundos
duracion_sim = 5 * 3600 

print("Propagando...")
# Esto devuelve un array de numpy: [tiempo, x, y, z]
trayectoria = prop.run(sat, duration_sec=duracion_sim, step_size=60)

print(f"¡Listo! Se generaron {trayectoria.shape[0]} puntos de datos.")

In [ ]:
--> moved to visualizer.py

# Llamada final:
figura = plot_ground_track(trayectoria)
figura.show()

In [ ]:
--> moved to visualizer.py

# Ejecución
fig_2d = graficar_2d_plotly(trayectoria)
fig_2d.show()

# From Celestrak

In [ ]:
from satCatalog import request_celestrak_data
from space_env import TLEHandler

In [ ]:
# Request Satellites to Celestrak
nusat_list = request_celestrak_data("NUSAT", "NAME")
nusat45=nusat_list[-1]
handler = TLEHandler.from_json(nusat45)

In [ ]:
handler.name.split()[0]

In [ ]:
# New Satellite
sat_name = handler.name.split()[0]
# 3. Get the state at the TLE's specific Epoch 
state = handler.get_state_at() 


In [ ]:
# Initialize GMAT Satellite and set the state
sat_nusat = Satellite(sat_name)
sat_nusat.set_cartesian(
    epoch_astropy=state['epoch'],
    pos=state['pos'],
    vel=state['vel']
)

In [ ]:
# 1. Definimos la duración (ejemplo: 1 órbita ~ 90 min = 5400 seg)
duration = 5400 
step = 60 # Guardar datos cada 60 segundos

# 2. Instanciamos tu Propagator
prop = Propagator(name="HighPrecisionProp")

# 3. Ejecutamos la propagación
# Esto devolverá un array de numpy con [tiempo, x, y, z]
trajectory_data = prop.run(sat_nusat, duration, step_size=step)

print(f"Propagación completada. Puntos calculados: {len(trajectory_data)}")
print(f"Última posición (km): {trajectory_data[-1, 1:]}")

In [ ]:
from visualizer import graficar_2d_plotly
# Ejecución
fig_2d = graficar_2d_plotly(trajectory_data)
fig_2d.show()